In [ ]:
def cond_indexing(a,BOOL,cond) :
    if BOOL=="<" :
        return([i for i in range(len(a)) if a[i] < cond])
    elif BOOL==">" :
        return([i for i in range(len(a)) if a[i] > cond])
    elif BOOL=="==" :
        return([i for i in range(len(a)) if a[i] == cond])
    elif BOOL=="<=" :
        return([i for i in range(len(a)) if a[i] <= cond])
    elif BOOL==">=" :
        return([i for i in range(len(a)) if a[i] >= cond])
    elif BOOL=="!=" :
        return([i for i in range(len(a)) if a[i] != cond])
    else :
        print "something wrong!"

In [ ]:
from PIL import Image as Img
import pandas as pd
import numpy as np
import glob as gb
import time
import re

t0 = time.clock()

match_str=r'\bradiomics_gtv\b|\bradiomics_gtv\b|\bRadiomics_gtv\b|\bradiomics_gtv2\b|\bradiomics_gtv_nw\b|\bradiomics_gtvr\b'

data_by_patient=[]
png_tag_by_patient=[]
patient_tag_by_patient=[]
patient_list=gb.glob("/home/ubuntu/data/example_extracted/*")
tumor_coords_by_patient=[]

length_patient=len(patient_list)
for j in xrange(0,10):#length_patient) : #len(patient_list)):
    print str(j)+' out of '+str(len(patient_list))
    
    char_list= str([line.rstrip('\n') for line in open(patient_list[j]+'/structures.dat')])
    matched=re.findall(match_str,char_list)
    
    indices = int([i for i, s in enumerate(char_list.split("|")) if matched[0] in s][0])+1
    #print patient_list[j]+'_'+str(indices)
    png_list=gb.glob(patient_list[j]+'/pngs/*.png')
    
    data_by_patient_by_png=[]
    png_tag=[]
    patient_tag=[]
    tumor_coords=[]
    length_pngs=len(png_list)
    
    for k in xrange(0,length_pngs) :
        png_loaded=Img.open(png_list[k],'r')
    
        data_by_patient_by_png.append(np.array(list(png_loaded.getdata())))
        png_tag_by_patient.append((png_list[k].split("pngs/")[1].split(".png")[0]))
        patient_tag_by_patient.append(patient_list[j].split('TC')[1])
        
        contour_path=patient_list[j]+'/contours/'+png_list[k].split('pngs/')[1].split('.png')[0]+'.'+str(indices)+'.dat'
        val = [i for i, s in enumerate((gb.glob(patient_list[j]+'/contours/*'))) if contour_path in s]
    
        if len(val)==0 :
            tumor_coords.append(['-1'])
        else :
            temp_a=[line.rstrip('\n') for line in open(contour_path)]
            if len(temp_a)>1 :
                tumor_coords.append( "@".join(temp_a))
            else :
                tumor_coords.append(temp_a)

    data_by_patient.append(np.vstack(data_by_patient_by_png))
    png_tag.append(np.vstack(png_tag_by_patient))
    patient_tag.append(np.vstack(patient_tag_by_patient))
    tumor_coords_by_patient.append(np.hstack(tumor_coords))
    
data_all=np.vstack(data_by_patient)
pngTag_all=np.vstack(png_tag)
patientTag_all=np.vstack(patient_tag)
tumor_coords_all=np.hstack(tumor_coords_by_patient)
tend = time.clock()

In [ ]:
no_tumor_index = cond_indexing(tumor_coords_all,"==",'-1')
yes_tumor_index = cond_indexing(tumor_coords_all,"!=",'-1')

In [ ]:
for k in xrange(0,len(yes_tumor_index)) :
    print str(k)+' out of '+str(len(yes_tumor_index))
    PATIENT_IND=str(patientTag_all[yes_tumor_index][k][0])
    PNG_IND=str(pngTag_all[yes_tumor_index][k][0])
    path=("/home/ubuntu/data/example_extracted/ANON_LUNG_TC"+PATIENT_IND+'/auxiliary/'+PNG_IND+'.dat')

    aux_data=[line.rstrip('\n') for line in open(path)]

    ind_0=[]
    ind_d=[]

    for i in xrange(0,len(aux_data)) :
        ind_0.append(aux_data[i].split('),')[0].split('(')[1] == '0020.0032' and i or 0 )
        ind_d.append(aux_data[i].split('),')[0].split('(')[1] == '0028.0030' and i or 0 )

    ind_0=cond_indexing(ind_0,"!=",0)[0]
    ind_d=cond_indexing(ind_d,"!=",0)[0]
    slice_x0=aux_data[ind_0].split('),')[1].split(',')[0]
    slice_y0=aux_data[ind_0].split('),')[1].split(',')[1]

    slice_dx=aux_data[ind_d].split('),')[1].split(',')[0]
    slice_dy=aux_data[ind_d].split('),')[1].split(',')[1]

    if len(tumor_coords_all[yes_tumor_index][k].split('@'))>1 :
        df=pd.DataFrame(np.array(tumor_coords_all[yes_tumor_index][k].split('@')[0].split(','),dtype='double').reshape(-1,3))  
    else :
        df=pd.DataFrame(np.array(tumor_coords_all[yes_tumor_index][k].split(','),dtype='double').reshape(-1,3))  

    df.columns = ["x", "y","z"]

    x1=(df.x-np.repeat(float(slice_x0),df.shape[0]))/np.repeat(float(slice_dx),df.shape[0])
    y1=(df.y-np.repeat(float(slice_y0),df.shape[0]))/np.repeat(float(slice_dy),df.shape[0])

    plt.figure(figsize=(5.12,5.12),num=None,facecolor='w',edgecolor='k')
    plt.xlim(0,500); plt.ylim(510,0)
    plt.fill(x1,y1, lw=1) 
    plt.axis('off')
    plt.savefig('temp.png', format='png', dpi=100)#,transparent=True)#,bbox_inches='tight')


    Img.open("/home/ubuntu/Notebooks/temp.png",'r').convert('L').save('temp_con.png')
    c=Img.open("/home/ubuntu/Notebooks/temp_con.png",'r')#
    pic_data=np.array(c.getdata())
    pic_data[cond_indexing((pic_data),"!=",255)]=0
    plt.show()
    PATIENT_IND=str(patientTag_all[yes_tumor_index][k][0])
    PNG_IND=str(pngTag_all[yes_tumor_index][k][0])

    orginal=Img.open("/home/ubuntu/data/example_extracted/ANON_LUNG_TC"+PATIENT_IND+'/pngs/'+PNG_IND+'.png','r')
    temp_orginal=np.array(orginal.getdata())
    temp_orginal[cond_indexing((pic_data),"!=",255)]=1
    temp_orginal[cond_indexing((pic_data),"==",255)]=0
    im=Img.new('L', (512, 512))
    im.putdata(temp_orginal)
    im.save('/home/ubuntu/data/target/yes/'+'target_'+PATIENT_IND+'_'+PNG_IND+'.png')
